In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
class DMoneyMarketSwap:    
    def __init__(self, evDate, effective, maturity, position, CCYComm, InitNPAComm, 
                 FinalNPAComm, CCYTerm, InitNPATerm, FinalNPATerm, FXSpot, h_discTSComm, 
                 h_discTSTerm):
        self._ValueDate = evDate
        self._EffectiveDate = effective
        self._Maturity = maturity        
        if position == 'Buy':          
            self._Position = 1.0
        else:
            self._Position = -1.0
        self._CCYComm = CCYComm
        self._InitNPAComm = InitNPAComm
        self._FinalNPAComm = FinalNPAComm
        self._CCYTerm = CCYTerm
        self._InitNPATerm = InitNPATerm
        self._FinalNPATerm = FinalNPATerm
        self._FXSpot = FXSpot
        self._H_discTSComm = h_discTSComm
        self._H_discTSTerm = h_discTSTerm
        
    def NPV(self):
        FinalDfComm = self._H_discTSComm.discount(self._Maturity)
        FinalDfTerm = self._H_discTSTerm.discount(self._Maturity)
        FinalPVComm = FinalDfComm * self._FinalNPAComm
        FinalPVTerm = FinalDfTerm * self._FinalNPATerm        
        if (self._ValueDate >= self._EffectiveDate):        
            self._Value = self._Position * (FinalPVComm * self._FXSpot - FinalPVTerm)
        else:
            InitDfComm = self._H_discTSComm.discount(self._EffectiveDate)
            InitDfTerm = self._H_discTSTerm.discount(self._EffectiveDate)
            InitPVComm = InitDfComm * self._InitNPAComm
            InitPVTerm = InitDfTerm * self._InitNPATerm        
            self._Value = self._Position * ((FinalPVComm * self._FXSpot - FinalPVTerm) 
                                            + (InitPVTerm - InitPVComm * self._FXSpot))
        return self._Value

In [4]:
PstBook = xl.load_workbook('Pst_MMS.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
TermCCY = 'TWD'
Type = 'MMS'
CommCCY = MktSheet.cell(3, 5).value
MTMDate = MktSheet.cell(1, 3).value
FXSpot = MktSheet.cell(3, 6).value

In [5]:
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
CommDC = ql.Actual360()
TermDC = ql.Actual365Fixed()
BDC = ql.Unadjusted
settlementDays = 0

In [6]:
CommZeroSP = MktSheet.cell(22, 6).value
CommZero3M = MktSheet.cell(23, 6).value
CommZero6M = MktSheet.cell(24, 6).value
CommZero1Y = MktSheet.cell(25, 6).value
print(CommZeroSP)

0.001459727696536763


In [7]:
DateSP = evDate
Date3M = Cal.advance(evDate, 3, ql.Months, BDC, False)
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date1Y = Cal.advance(evDate, 1, ql.Years, BDC, False)
print(DateSP)

June 30th, 2021


In [8]:
TermZeroSP = MktSheet.cell(22, 12).value
TermZero3M = MktSheet.cell(23, 12).value
TermZero6M = MktSheet.cell(24, 12).value
TermZero1Y = MktSheet.cell(25, 12).value
print(TermZeroSP)

-0.00392904297456824


In [9]:
ListDate = list([DateSP, Date3M, Date6M, Date1Y])
ListCommRate = list([CommZeroSP, CommZero3M, CommZero6M, CommZero1Y])
ListTermRate = list([TermZeroSP, TermZero3M, TermZero6M, TermZero1Y])
ListDate
ListCommRate
ListTermRate

[-0.00392904297456824,
 -0.003929042973135284,
 -0.0006520395000015291,
 0.001329938573603508]

In [10]:
VecDate = ql.DateVector(ListDate)
VecCommRate = ql.DoubleVector(ListCommRate)
VecTermRate = ql.DoubleVector(ListTermRate)

In [11]:
TSCommZero = ql.ZeroCurve(VecDate, VecCommRate, CommDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSCommZero = ql.YieldTermStructureHandle(TSCommZero)
h_TSCommZero_0 = ql.YieldTermStructureHandle(TSCommZero)
TSTermZero = ql.ZeroCurve(VecDate, VecTermRate, TermDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSTermZero = ql.YieldTermStructureHandle(TSTermZero)
h_TSTermZero_0 = ql.YieldTermStructureHandle(TSTermZero)

In [12]:
NoCurv = len(ListCommRate)
print(NoCurv)

4


In [13]:
ListListCommRate = list()
ListListCommRate.append(ListCommRate)

In [14]:
ListListCommRate

[[0.001459727696536763,
  0.001459727696619909,
  0.002038942990682572,
  0.002656419500050458]]

In [15]:
for i in range(1, NoCurv):
    NewList = list()
    NewList.append(ListCommRate[0])
    for j in range(1, NoCurv):
        if(i==j):
            NewList.append(ListCommRate[j] + 0.0001)
        else:
            NewList.append(ListCommRate[j])
    ListListCommRate.append(NewList)

In [16]:
ListListCommRate

[[0.001459727696536763,
  0.001459727696619909,
  0.002038942990682572,
  0.002656419500050458],
 [0.001459727696536763,
  0.001559727696619909,
  0.002038942990682572,
  0.002656419500050458],
 [0.001459727696536763,
  0.001459727696619909,
  0.002138942990682572,
  0.002656419500050458],
 [0.001459727696536763,
  0.001459727696619909,
  0.002038942990682572,
  0.002756419500050458]]

In [17]:
ListVecCommRate = list()
ListTSCommRate = list()
for i in range(NoCurv):
    AVecCommRate = ql.DoubleVector(ListListCommRate[i])
    ListVecCommRate.append(AVecCommRate)
    ATSCommRate = ql.ZeroCurve(VecDate, AVecCommRate, CommDC, Cal, ql.Linear(), 
                               ql.Continuous, ql.Annual)
    ListTSCommRate.append(ATSCommRate)

In [18]:
ListListTermRate = list()
ListListTermRate.append(ListTermRate)

In [19]:
ListListTermRate

[[-0.00392904297456824,
  -0.003929042973135284,
  -0.0006520395000015291,
  0.001329938573603508]]

In [20]:
for i in range(1, NoCurv):
    NewList = list()
    NewList.append(ListTermRate[0])
    for j in range(1, NoCurv):
        if(i==j):
            NewList.append(ListTermRate[j] + 0.0001)
        else:
            NewList.append(ListTermRate[j])
    ListListTermRate.append(NewList)

In [21]:
ListListTermRate

[[-0.00392904297456824,
  -0.003929042973135284,
  -0.0006520395000015291,
  0.001329938573603508],
 [-0.00392904297456824,
  -0.003829042973135284,
  -0.0006520395000015291,
  0.001329938573603508],
 [-0.00392904297456824,
  -0.003929042973135284,
  -0.000552039500001529,
  0.001329938573603508],
 [-0.00392904297456824,
  -0.003929042973135284,
  -0.0006520395000015291,
  0.001429938573603508]]

In [22]:
ListVecTermRate = list()
ListTSTermRate = list()
for i in range(NoCurv):
    AVecTermRate = ql.DoubleVector(ListListTermRate[i])
    ListVecTermRate.append(AVecTermRate)
    ATSTermRate = ql.ZeroCurve(VecDate, AVecTermRate, TermDC, Cal, ql.Linear(), 
                               ql.Continuous, ql.Annual)
    ListTSTermRate.append(ATSTermRate)

In [23]:
CurvUpListListCommRate = list()
CurvUpListListCommRate.append(ListCommRate)
for i in range(1, NoCurv):
    NewList = list()
    NewList.append(ListCommRate[0])
    for j in range(1, NoCurv):
        if(i==j):
            NewList.append(ListCommRate[j] + 0.017)
        else:
            NewList.append(ListCommRate[j])
    CurvUpListListCommRate.append(NewList)

In [24]:
CurvDnListListCommRate = list()
CurvDnListListCommRate.append(ListCommRate)
for i in range(1, NoCurv):
    NewList = list()
    NewList.append(ListCommRate[0])
    for j in range(1, NoCurv):
        if(i==j):
            NewList.append(ListCommRate[j] - 0.017)
        else:
            NewList.append(ListCommRate[j])
    CurvDnListListCommRate.append(NewList)

In [25]:
CurvUpListListTermRate = list()
CurvUpListListTermRate.append(ListTermRate)
for i in range(1, NoCurv):
    NewList = list()
    NewList.append(ListTermRate[0])
    for j in range(1, NoCurv):
        if(i==j):
            NewList.append(ListTermRate[j] + 0.017)
        else:
            NewList.append(ListTermRate[j])
    CurvUpListListTermRate.append(NewList)

In [26]:
CurvDnListListTermRate = list()
CurvDnListListTermRate.append(ListTermRate)
for i in range(1, NoCurv):
    NewList = list()
    NewList.append(ListTermRate[0])
    for j in range(1, NoCurv):
        if(i==j):
            NewList.append(ListTermRate[j] - 0.017)
        else:
            NewList.append(ListTermRate[j])
    CurvDnListListTermRate.append(NewList)

In [27]:
MMSSheet = PstBook['MMS']

In [28]:
start_row = 2
end_row = 4
start_col = 0
end_col = 10
recordcount = end_col - start_col + 1
ListMMS = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in MMSSheet[i][start_col:end_col+1]]
    ListMMS.append(row)
    print(row)

['MMS001', datetime.datetime(2021, 5, 19, 0, 0), datetime.datetime(2021, 5, 21, 0, 0), datetime.datetime(2021, 9, 21, 0, 0), 'Buy', 'USD', 1000000, 1000000, 'TWD', 28000000, 27900000]
['MMS002', datetime.datetime(2021, 6, 13, 0, 0), datetime.datetime(2021, 6, 15, 0, 0), datetime.datetime(2021, 12, 15, 0, 0), 'Sell', 'USD', 1000000, 1000000, 'TWD', 27900000, 27800000]
['MMS003', datetime.datetime(2021, 6, 21, 0, 0), datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2022, 5, 23, 0, 0), 'Buy', 'USD', 1000000, 1000000, 'TWD', 27850000, 27700000]


In [29]:
MMSBook = list()
for i in range(len(ListMMS)):
    row = ListMMS[i]
    record = list()
    record.append(row[0])
    record.append(DatetimeToDate(row[1]))
    record.append(DatetimeToDate(row[2]))
    record.append(DatetimeToDate(row[3]))
    record.append(row[4])
    record.append(row[5])
    record.append(row[6])
    record.append(row[7])
    record.append(row[8])
    record.append(row[9])
    record.append(row[10])
    MMSBook.append(record)

In [30]:
MMSBook

[['MMS001',
  Date(19,5,2021),
  Date(21,5,2021),
  Date(21,9,2021),
  'Buy',
  'USD',
  1000000,
  1000000,
  'TWD',
  28000000,
  27900000],
 ['MMS002',
  Date(13,6,2021),
  Date(15,6,2021),
  Date(15,12,2021),
  'Sell',
  'USD',
  1000000,
  1000000,
  'TWD',
  27900000,
  27800000],
 ['MMS003',
  Date(21,6,2021),
  Date(23,6,2021),
  Date(23,5,2022),
  'Buy',
  'USD',
  1000000,
  1000000,
  'TWD',
  27850000,
  27700000]]

In [31]:
SenSheet = PstBook['GIRR']
for i in range(0, len(MMSBook)):
    h_TSCommZero = h_TSCommZero_0
    h_TSTermZero = h_TSTermZero_0
    
    ListValue = list()
    record = MMSBook[i]
    tradeDate = record[1]
    effectiveDate = record[2]
    maturityDate = record[3]
    position = record[4]
    CommCCY = record[5]    
    InitCommAmt = record[6]
    FinalCommAmt = record[7]
    TermCCY = record[8]    
    InitTermAmt = record[9]
    FinalTermAmt = record[10]
    MMS = DMoneyMarketSwap(evDate, effectiveDate, maturityDate, position, CommCCY, 
                           InitCommAmt, FinalCommAmt, TermCCY, InitTermAmt,
                           FinalTermAmt, FXSpot, h_TSCommZero, h_TSTermZero)    
    Value = MMS.NPV()        
    ListValue.append(Value)
    
    for j in range(1, NoCurv):
        h_TSCommZero = ql.YieldTermStructureHandle(ListTSCommRate[j])
        MMS = DMoneyMarketSwap(evDate, effectiveDate, maturityDate, position, CommCCY,
                               InitCommAmt, FinalCommAmt, TermCCY, InitTermAmt,
                               FinalTermAmt, FXSpot, h_TSCommZero, h_TSTermZero)
        Value = MMS.NPV()                
        ListValue.append(Value)
        SenSheet.cell(i+5, 1).value = record[0]
        SenSheet.cell(i+5, 2).value = ListValue[0]
        SenSheet.cell(i+5, 3).value = CommCCY

    ListDelta = list()
    for j in range(1, len(ListValue)):
        ListDelta.append((ListValue[j] - ListValue[0])*10000)
    
    for j in range(0, len(ListDelta)):    
        SenSheet.cell(i+5, j+5).value = ListDelta[j]
    
    ListVecCommRate = list()
    CurvUpListTSCommRate = list()
    for j in range(NoCurv):
        AVecCommRate = ql.DoubleVector(CurvUpListListCommRate[j])
        ListVecCommRate.append(AVecCommRate)
        ATSCommRate = ql.ZeroCurve(VecDate, AVecCommRate, CommDC, Cal, ql.Linear(), 
                                   ql.Continuous, ql.Annual)
        CurvUpListTSCommRate.append(ATSCommRate)
                
    ListVecCommRate = list()
    CurvDnListTSCommRate = list()
    for j in range(NoCurv):
        AVecCommRate = ql.DoubleVector(CurvDnListListCommRate[j])
        ListVecCommRate.append(AVecCommRate)
        ATSCommRate = ql.ZeroCurve(VecDate, AVecCommRate, CommDC, Cal, ql.Linear(), 
                                   ql.Continuous, ql.Annual)
        CurvDnListTSCommRate.append(ATSCommRate)
        
    CurvUpListHTSCommRate = list()
    for j in range(NoCurv):
        CurvUpListHTSCommRate.append(ql.YieldTermStructureHandle(CurvUpListTSCommRate[j]))
        
    CurvDnListHTSCommRate = list()
    for j in range(NoCurv):
        CurvDnListHTSCommRate.append(ql.YieldTermStructureHandle(CurvDnListTSCommRate[j]))
        
    CurvUpValue = list()
    for j in range(NoCurv):
        h_TSCommZero = CurvUpListHTSCommRate[j]
        MMS = DMoneyMarketSwap(evDate, effectiveDate, maturityDate, position, CommCCY,
                               InitCommAmt, FinalCommAmt, TermCCY, InitTermAmt,
                               FinalTermAmt, FXSpot, h_TSCommZero, h_TSTermZero)
        Value = MMS.NPV()                
        CurvUpValue.append(Value)
    
    CurvDnValue = list()
    for j in range(NoCurv):
        h_TSCommZero = CurvDnListHTSCommRate[j]
        MMS = DMoneyMarketSwap(evDate, effectiveDate, maturityDate, position, CommCCY,
                               InitCommAmt, FinalCommAmt, TermCCY, InitTermAmt,
                               FinalTermAmt, FXSpot, h_TSCommZero, h_TSTermZero)
        Value = MMS.NPV()                
        CurvDnValue.append(Value)
        
    CurvatureUp = list()
    ShiftUpValue = list()
    DeltaChange = list()
    for j in range(1, NoCurv):
        ShiftUpValue.append(CurvUpValue[j]-CurvUpValue[0])        
        CurvatureUp.append(-((CurvUpValue[j]-CurvUpValue[0]) - 0.017 * ListDelta[j-1]))
                
    for j in range(0, len(CurvatureUp)):
        SenSheet.cell(i+5, j+17).value = CurvatureUp[j]

    CurvatureDn = list()
    ShiftDnValue = list()
    for j in range(1, NoCurv):
        ShiftDnValue.append(CurvDnValue[j]-CurvDnValue[0])
        CurvatureDn.append(-((CurvDnValue[j]-CurvDnValue[0]) + 0.017 * ListDelta[j-1]))

    for j in range(0, len(CurvatureDn)):
        SenSheet.cell(i+5, j+29).value = CurvatureDn[j]

PstBook.save('Pst_MMS.xlsx')
PstBook.close()